In [1]:
import os

from geopandas import GeoDataFrame

from pyspark import SparkConf, SparkFiles
from pyspark.sql import SparkSession
from pyspark.sql.functions import broadcast, pandas_udf, PandasUDFType, udf, col, rank
from pyspark.sql.window import Window
from pyspark.sql.types import StringType

from sedona.utils import KryoSerializer, SedonaKryoRegistrator
from sedona.utils.adapter import Adapter
from sedona.register import SedonaRegistrator

### Create SparkSession

In [2]:
#Create Spark configuration to run Spark jobs with Sedona capabilities
conf = SparkConf()
conf.set("spark.serializer", KryoSerializer.getName)
conf.set("spark.kryo.registrator", SedonaKryoRegistrator.getName)
conf.set('spark.jars.packages',
           'org.apache.sedona:sedona-python-adapter-3.0_2.12:1.0.0-incubating,'
           'org.datasyslab:geotools-wrapper:geotools-24.0')

In [ ]:
#Additional configuration to have access to S3 bucket
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.3.3,com.amazonaws:aws-java-sdk:1.12.258')
conf.set("spark.hadoop.fs.s3a.access.key", os.getenv('AWS_S3_KEY'))
conf.set("spark.hadoop.fs.s3a.secret.key", os.getenv('AWS_S3_SECRET'))

In [3]:
os.environ['PYSPARK_DRIVER_PYTHON'] = "/opt/conda/envs/venv36/bin/python"
spark = SparkSession.builder.master("spark://spark:7077"). \
    config(conf=conf). \
    appName('OpenSky_app'). \
    getOrCreate()

In [4]:
#Add spatial functionality to the SparkSession
SedonaRegistrator.registerAll(spark)

True

### Load states boundaries dataset

In [5]:
#Load data from local file to geoDataFrame
geo_admin_url = 'admin1-us.geojson'
gdf_states = GeoDataFrame.from_file(geo_admin_url)

#Create pySpark dataframe and view
spark_states_df = spark.createDataFrame(gdf_states)
spark_states_df.createOrReplaceTempView('states')

/opt/conda/envs/venv36/lib/python3.6/site-packages/geopandas/array.py:85: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  aout[:] = out
/opt/conda/envs/venv36/lib/python3.6/site-packages/geopandas/array.py:85: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  aout[:] = out


In [6]:
spark_states_df.printSchema()

root
 |-- name: string (nullable = true)
 |-- country: string (nullable = true)
 |-- ISO3166-1-Alpha-3: string (nullable = true)
 |-- state_code: string (nullable = true)
 |-- id: string (nullable = true)
 |-- geometry: geometry (nullable = true)



In [7]:
#Inspect view
spark.sql(
    """
    SELECT * FROM states ORDER BY name 
    """
).show(5)

+----------+--------------------+-----------------+----------+--------+--------------------+
|      name|             country|ISO3166-1-Alpha-3|state_code|      id|            geometry|
+----------+--------------------+-----------------+----------+--------+--------------------+
|   Alabama|United States of ...|              USA|        AL|USA-3541|POLYGON ((-85.054...|
|    Alaska|United States of ...|              USA|        AK|USA-3563|MULTIPOLYGON (((-...|
|   Arizona|United States of ...|              USA|        AZ|USA-3520|POLYGON ((-109.04...|
|  Arkansas|United States of ...|              USA|        AR|USA-3528|POLYGON ((-89.662...|
|California|United States of ...|              USA|        CA|USA-3521|POLYGON ((-114.35...|
+----------+--------------------+-----------------+----------+--------+--------------------+
only showing top 5 rows



### Create dataframe from local data

In [8]:
df=spark.read.csv('/opt/bitnami/spark/temp/states_2022-01-03-00.csv',inferSchema =True, header = True)
# df=spark.read.csv('/opt/bitnami/spark/temp/',inferSchema =True, header = True)

### Create dataframe from S3 located data

In [ ]:
df=spark.read.csv('s3a://avia-data-2022-03-01/data/',inferSchema =True, header = True)

### Create UDF to transform lat/long to shapely geometry

In [9]:
@udf(returnType=StringType())
def create_WKT(lat, lon):
    wkt_point = f'POINT({lon} {lat})'
    return wkt_point

### Create UDF to transforn angle of heading into named direction

In [30]:
@udf(returnType=StringType())
def create_direction(angle):
    if angle:
        if 337.5 < angle or angle < 45: return "North"
        if 67.5 > angle > 22.5 : return "Northeast"
        if 112.5 > angle > 67.5 : return "East"
        if 157.5 > angle > 112.5 : return "Southeast"
        if 202.5 > angle > 157.5 : return "South"
        if 247.5 > angle > 202.5 : return "Southwest"
        if 292.5 > angle > 247.5 : return "West"
        if 337.5 > angle > 292.5 : return "Northwest"
    return None

### Discover data schema

In [11]:
df.printSchema()

root
 |-- time: integer (nullable = true)
 |-- icao24: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- velocity: double (nullable = true)
 |-- heading: double (nullable = true)
 |-- vertrate: double (nullable = true)
 |-- callsign: string (nullable = true)
 |-- onground: boolean (nullable = true)
 |-- alert: boolean (nullable = true)
 |-- spi: boolean (nullable = true)
 |-- squawk: integer (nullable = true)
 |-- baroaltitude: double (nullable = true)
 |-- geoaltitude: double (nullable = true)
 |-- lastposupdate: double (nullable = true)
 |-- lastcontact: double (nullable = true)



# A total number of flying vehicles in particular dataset

In [12]:
unicue_vehicles = df.select('icao24').distinct().count()
print(f'Number of unique vehicles is {unicue_vehicles}')

Number of unique vehicles is 10508


### Modify DataFrame and add a column with WKT coordinates, shapely geometry and direction

In [31]:
#Filterout null data and vehicles ion the ground
df.dropna(subset=("icao24", "lat", "lon")). \
    filter(col('onground') == False). \
    withColumn('geometry_wkt', create_WKT(col('lat'), col('lon'))). \
    withColumn('heading_str', create_direction(col('heading'))). \
    createOrReplaceTempView('points')

In [32]:
merged_df = spark.sql(
    """
    SELECT *
    FROM (SELECT *, ST_GeomFromWKT(geometry_wkt) as geometry 
            FROM points) AS p, states as S
    WHERE ST_Intersects(p.geometry, s.geometry)
    """
)
merged_df.createOrReplaceTempView('merged_view')

In [15]:
merged_df.printSchema()

root
 |-- time: integer (nullable = true)
 |-- icao24: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- velocity: double (nullable = true)
 |-- heading: double (nullable = true)
 |-- vertrate: double (nullable = true)
 |-- callsign: string (nullable = true)
 |-- onground: boolean (nullable = true)
 |-- alert: boolean (nullable = true)
 |-- spi: boolean (nullable = true)
 |-- squawk: integer (nullable = true)
 |-- baroaltitude: double (nullable = true)
 |-- geoaltitude: double (nullable = true)
 |-- lastposupdate: double (nullable = true)
 |-- lastcontact: double (nullable = true)
 |-- geometry_wkt: string (nullable = true)
 |-- heading_str: string (nullable = true)
 |-- geometry: geometry (nullable = false)
 |-- name: string (nullable = true)
 |-- country: string (nullable = true)
 |-- ISO3166-1-Alpha-3: string (nullable = true)
 |-- state_code: string (nullable = true)
 |-- id: string (nullable = true)
 |-- geometry: geometry (nullabl

### Find top values of vertical speed, speed of ascending and descending, and the highest flight in the all US teritory

In [17]:
TOP_values_df = spark.sql(
    """
    SELECT DISTINCT "fastest vertival" as nomination, icao24, name as state, mv.velocity as value
    FROM merged_view as mv
    INNER JOIN    
        (SELECT max(mv) as velocity
        FROM (SELECT icao24, max(velocity) as mv
                FROM merged_view
                GROUP BY icao24) as mv) as mx
                ON mv.velocity = mx.velocity
    UNION
    SELECT DISTINCT "fastest ascending" as nomination, icao24, name as state, mv.vertrate as value
    FROM merged_view as mv
    INNER JOIN    
        (SELECT max(mv) as vertrate
        FROM (SELECT icao24, max(vertrate) as mv
                FROM merged_view
                GROUP BY icao24) as mv) as mx
                ON mv.vertrate = mx.vertrate
    UNION
    SELECT DISTINCT "fastest descending" as nomination, icao24, name as state, mv.vertrate as value
    FROM merged_view as mv
    INNER JOIN    
        (SELECT min(mv) as vertrate
        FROM (SELECT icao24, min(vertrate) as mv
                FROM merged_view
                GROUP BY icao24) as mv) as mx
                ON mv.vertrate = mx.vertrate
    UNION
    SELECT DISTINCT "highest flight" as nomination, icao24, name as state, mv.geoaltitude as value
    FROM merged_view as mv
    INNER JOIN    
        (SELECT max(geoalt) as geoalt
        FROM (SELECT icao24, max(geoaltitude) as geoalt
                FROM merged_view
                GROUP BY icao24) as mv) as mx
                ON mv.geoaltitude = mx.geoalt
    """
)

In [20]:
pd_TOP_values_df = TOP_values_df.toPandas()

In [21]:
pd_TOP_values_df

,nomination,icao24,state,value
0,fastest descending,a92a7c,California,-95.91040
1,fastest ascending,a1e9a0,Indiana,165.81120
2,fastest vertival,a43ca8,Alabama,554.63386
3,fastest ascending,a585e4,Louisiana,165.81120
4,highest flight,a8793d,South Carolina,38221.92000


### Find the most common side of the world as direction of flight

In [33]:
#The most common direction
spark.sql(
    """
    SELECT heading_str, count (*) as quantity
    FROM merged_view
    WHERE heading_str IS NOT NULL
    GROUP BY heading_str
    ORDER BY quantity DESC
    """
).show()

+-----------+--------+
|heading_str|quantity|
+-----------+--------+
|       West|  240066|
|      North|  204990|
|       East|  185085|
|  Northwest|  151408|
|  Southwest|  127541|
|      South|  123995|
|  Southeast|  109286|
|  Northeast|   68357|
+-----------+--------+



### Find top values of vertical speed, speed of ascending and descending, and the highest flight in each state

In [34]:
#Fastest vehicle
spark.sql(
    """
    SELECT DISTINCT "fastest vertival" as nomination, name as state, icao24,  mv.velocity as value
    FROM merged_view as mv
    INNER JOIN    
        (SELECT state, max(mv) as velocity
        FROM (SELECT name as state, icao24, max(velocity) as mv
                FROM merged_view
                GROUP BY name, icao24) as mv
        GROUP BY state) as mx
    ON mv.velocity = mx.velocity and mv.name = mx.state
    """
).createOrReplaceTempView('fastest_State')

In [35]:
#Fastest ascending
spark.sql(
    """
    SELECT DISTINCT "fastest ascending" as nomination, name as state, icao24,  mv.vertrate as value
        FROM merged_view as mv
        INNER JOIN    
            (SELECT state, max(mv) as vertrate
            FROM (SELECT name as state, icao24, max(vertrate) as mv
                    FROM merged_view
                    GROUP BY name, icao24) as mv
            GROUP BY state) as mx
        ON mv.vertrate = mx.vertrate and mv.name = mx.state
    """
).createOrReplaceTempView('fastest_ASC')

In [36]:
#Fastest descending
spark.sql(
    """
    SELECT DISTINCT "fastest ascending" as nomination, name as state, icao24,  mv.vertrate as value
        FROM merged_view as mv
        INNER JOIN    
            (SELECT state, min(mv) as vertrate
            FROM (SELECT name as state, icao24, min(vertrate) as mv
                    FROM merged_view
                    GROUP BY name, icao24) as mv
            GROUP BY state) as mx
        ON mv.vertrate = mx.vertrate and mv.name = mx.state
    """
).createOrReplaceTempView('fastest_DESC')

In [37]:
#Highest flight
spark.sql(
    """    
    SELECT DISTINCT "highest flight" as nomination, name as state, icao24, mv.geoaltitude as value
    FROM merged_view as mv
    INNER JOIN    
        (SELECT state, max(geoalt) as geoalt
        FROM (SELECT name as state, icao24, max(geoaltitude) as geoalt
                FROM merged_view
                GROUP BY name, icao24) as mv
        GROUP BY state) as mx
                ON mv.geoaltitude = mx.geoalt and mv.name = mx.state
    """
).createOrReplaceTempView('highest')

In [38]:
state_result_df = spark.sql(
    """
    SELECT 
      fa.state, 
      fa.highest_altitude, 
      fa.highest_plane, 
      fa.fastest_desc, 
      fa.fastest_dc_plain, 
      fa.fastest_asc, 
      fa.fastest_asc_pain, 
      fs.value as highest_velocity, 
      fs.icao24 as fastest_plain 
    FROM 
      (
        SELECT 
          fd.state, 
          fd.highest_altitude, 
          fd.highest_plane, 
          fd.fastest_desc, 
          fd.fastest_dc_plain, 
          fa.value as fastest_asc, 
          fa.icao24 as fastest_asc_pain 
          FROM
              (
            SELECT 
              h.state as state, 
              h.highest_altitude, 
              h.highest_plane, 
              fd.value as fastest_desc, 
              fd.icao24 as fastest_dc_plain 
            FROM 
              (
                SELECT 
                  s.name as state, 
                  h.value as highest_altitude, 
                  h.icao24 as highest_plane 
                FROM 
                  states as s 
                  LEFT JOIN highest as h ON h.state = s.name 
                ORDER BY 
                  s.name
              ) as h 
              LEFT JOIN fastest_DESC as fd ON h.state = fd.state
          ) as fd 
          LEFT JOIN fastest_ASC as fa ON fd.state = fa.state
      ) as fa 
      LEFT JOIN fastest_State as fs ON fa.state = fs.state
    """
)

In [39]:
#Store results as csv
pd_state_result_df = state_result_df.toPandas()

In [40]:
pd_state_result_df

,state,highest_altitude,highest_plane,fastest_desc,fastest_dc_plain,fastest_asc,fastest_asc_pain,highest_velocity,fastest_plain
0,Utah,14516.10,a17e87,-28.28544,a88d06,33.16224,a66530,288.643888,424b61
1,Hawaii,NaN,None,NaN,None,NaN,None,NaN,None
2,Minnesota,13548.36,a69fcb,-27.31008,a57238,29.26080,aca2be,296.877427,abba93
3,Ohio,26502.36,aa0b10,-20.48256,a641b6,44.54144,a890c7,322.749964,a11ba0
4,Arkansas,35852.10,a384f2,-18.20672,a7268a,26.65984,ac498c,291.595373,aa5631
5,Oregon,13632.18,a9f0ce,-25.35936,a6ce5d,20.15744,a55ef3,304.109949,ab0096
6,Texas,37696.14,a51e24,-32.83712,780b27,34.46272,a7c524,302.665379,a448f8
7,North Dakota,11955.78,a57362,-7.47776,a36200,18.20672,ac8d5c,301.309636,abba93
8,Pennsylvania,34998.66,a90bdb,-27.96032,a41b89,33.48736,a06a02,331.043415,ab3d41
9,Connecticut,12230.10,c02eb9,-17.88160,aa88d2,38.36416,a2badf,346.995052,a05646


### Find the most common direction by each state

In [42]:
#The most common direction
windowSpec = Window.partitionBy("state").orderBy("quantity")

df_heading = spark.sql(
    """
        SELECT name as state, heading_str, count (*) as quantity
            FROM merged_view
            WHERE heading_str IS NOT NULL
            GROUP BY heading_str, name
            ORDER BY quantity DESC
    """
).withColumn("rank",rank().over(windowSpec)). \
    filter(col('rank') == 1). \
    select('state', 'heading_Str')

In [43]:
pd_df_heading = df_heading.toPandas()

In [44]:
pd_df_heading

,state,heading_Str
0,Utah,Northeast
1,Minnesota,South
2,Ohio,Southwest
3,Arkansas,South
4,Oregon,Northeast
5,Texas,Northeast
6,North Dakota,Northeast
7,Pennsylvania,Northeast
8,Connecticut,Southeast
9,Nebraska,North
